In [1]:
import os
import shutil
import cv2
import pandas as pd
import numpy as np
from PIL import Image


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision import models, transforms
from skimage.exposure import match_histograms
import random
from skimage.io import imread
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [3]:
import isCarRealModel as m

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
def predict_image(image_path, model, device):
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        return None, None

    # Ensure model is in evaluation mode
    model.eval()

    # Must match input size expected by the model
    transform = transforms.Compose([
        transforms.Resize((480, 640)),
        transforms.ToTensor(),
    ])

    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)
    image = image.to(device)

    # Make prediction
    with torch.no_grad():
        output = model(image)
        prob = torch.sigmoid(output).item()
        prediction = 1 if prob >= 0.5 else 0

    # Output label_map = {'not': 0, 'real': 1}
    # print(f"Prediction: {'Not Real (0)' if prediction == 0 else 'Real (1)'} | Probability: {prob:.4f}")
    return prediction, prob

In [6]:
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tqdm.auto import tqdm

# Define label map based on folder names
label_map = {'spoof': 0, 'real': 1}

# Root directory containing class subfolders
image_dir = './final real not real random'


def evaluate_model_predictions(model, device, predict_image, label_map, image_base_path="./final real not real random"):

    # Parameters:
    # - model: The model to use for prediction.
    # - device
    # - predict_image (function)
    # - label_map (dict)
    # - image_base_path (str): Base path to be prepended to image paths in the DataFrame.

    # Store predictions and true labels
    y_true = []
    y_pred = []

    # Loop through both class folders
    for label_name, label_value in label_map.items():
        folder_path = os.path.join(image_dir, label_name)
        image_list = [img for img in os.listdir(folder_path) if img.lower().endswith(('.jpg', '.png', '.jpeg'))]

        for image_name in tqdm(image_list, desc=f"Processing {label_name}"):
            image_path = os.path.join(folder_path, image_name)
            pred, prob = predict_image(image_path, model, device)
            if pred is not None:
                y_true.append(label_value)
                y_pred.append(pred)

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=0)
    recall = recall_score(y_true, y_pred,pos_label=0)
    f1 = f1_score(y_true, y_pred,pos_label=0)
    specificity = recall_score(y_true, y_pred, pos_label=1)
    conf_matrix = confusion_matrix(y_true, y_pred)
    class_report = classification_report(y_true, y_pred, target_names=[str(name) for name in label_map.values()])



    # Print results
    print("Classification Report:\n", class_report)
    print("Confusion Matrix:\n", conf_matrix)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")

In [7]:
%cd ..

/content/drive/My Drive


# resnet-18

In [8]:
# Instantiate model
model = m.IsCarRealModel_resnet18(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_resnet18.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 214MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.74      1.00      0.85       600
           1       0.99      0.65      0.79       600

    accuracy                           0.82      1200
   macro avg       0.87      0.82      0.82      1200
weighted avg       0.87      0.82      0.82      1200

Confusion Matrix:
 [[598   2]
 [209 391]]
Accuracy: 0.8242
Precision: 0.7410
Recall: 0.9967
Specificity: 0.6517
F1 Score: 0.8500


# resnet-50

In [9]:
# Instantiate model
model = m.IsCarRealModel_resnet50(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_resnet50.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 225MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.99      0.85       600
           1       0.98      0.65      0.79       600

    accuracy                           0.82      1200
   macro avg       0.86      0.82      0.82      1200
weighted avg       0.86      0.82      0.82      1200

Confusion Matrix:
 [[594   6]
 [208 392]]
Accuracy: 0.8217
Precision: 0.7406
Recall: 0.9900
Specificity: 0.6533
F1 Score: 0.8474


# resnet-101

In [10]:
# Instantiate model
model = m.IsCarRealModel_resnet101(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_resnet101.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 237MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.99      0.85       600
           1       0.98      0.67      0.80       600

    accuracy                           0.83      1200
   macro avg       0.87      0.83      0.83      1200
weighted avg       0.87      0.83      0.83      1200

Confusion Matrix:
 [[593   7]
 [196 404]]
Accuracy: 0.8308
Precision: 0.7516
Recall: 0.9883
Specificity: 0.6733
F1 Score: 0.8539


# resnet-152

In [11]:
# Instantiate model
model = m.IsCarRealModel_resnet152(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_resnet152.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:15<00:00, 15.9MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.99      0.86       600
           1       0.99      0.69      0.81       600

    accuracy                           0.84      1200
   macro avg       0.88      0.84      0.84      1200
weighted avg       0.88      0.84      0.84      1200

Confusion Matrix:
 [[596   4]
 [186 414]]
Accuracy: 0.8417
Precision: 0.7621
Recall: 0.9933
Specificity: 0.6900
F1 Score: 0.8625


# mobilenet_v2

In [12]:
# Instantiate model
model = m.IsCarRealModel_mobilenet_v2(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_.mobilenet_v2.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 75.3MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.99      0.78       600
           1       0.99      0.45      0.61       600

    accuracy                           0.72      1200
   macro avg       0.81      0.72      0.70      1200
weighted avg       0.81      0.72      0.70      1200

Confusion Matrix:
 [[596   4]
 [332 268]]
Accuracy: 0.7200
Precision: 0.6422
Recall: 0.9933
Specificity: 0.4467
F1 Score: 0.7801


# mobilenet_v3_large

In [13]:
# Instantiate model
model = m.IsCarRealModel_mobilenet_v3_large(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_mobilenet_v3_large.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 218MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.73      1.00      0.85       600
           1       1.00      0.63      0.78       600

    accuracy                           0.82      1200
   macro avg       0.87      0.82      0.81      1200
weighted avg       0.87      0.82      0.81      1200

Confusion Matrix:
 [[600   0]
 [220 380]]
Accuracy: 0.8167
Precision: 0.7317
Recall: 1.0000
Specificity: 0.6333
F1 Score: 0.8451


# vgg16

In [14]:
# Instantiate model
model = m.IsCarRealModel_vgg16(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_vgg16.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 237MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.99      0.82       600
           1       0.99      0.57      0.73       600

    accuracy                           0.78      1200
   macro avg       0.85      0.78      0.77      1200
weighted avg       0.85      0.78      0.77      1200

Confusion Matrix:
 [[597   3]
 [257 343]]
Accuracy: 0.7833
Precision: 0.6991
Recall: 0.9950
Specificity: 0.5717
F1 Score: 0.8212


# vgg19

In [15]:
# Instantiate model
model = m.IsCarRealModel_vgg19(pretrained=True)
model = model.to(device)

# Load best model (map to correct device)
model.load_state_dict(torch.load('best_car_real_model_vgg19.pth', map_location=device))
model.to(device)

#Evalauate
evaluate_model_predictions(model, device, predict_image, label_map)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 233MB/s]


Processing spoof:   0%|          | 0/600 [00:00<?, ?it/s]

Processing real:   0%|          | 0/600 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.99      0.82       600
           1       0.99      0.57      0.73       600

    accuracy                           0.78      1200
   macro avg       0.84      0.78      0.77      1200
weighted avg       0.84      0.78      0.77      1200

Confusion Matrix:
 [[596   4]
 [256 344]]
Accuracy: 0.7833
Precision: 0.6995
Recall: 0.9933
Specificity: 0.5733
F1 Score: 0.8209
